Matt Rock

Importing the different libraries and setting up the Twitter API.

In [1]:
import tweepy
from datetime import datetime
import pandas as pd
import time
%run ~/twitter_credentials.py
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)
api = tweepy.API(auth)

Initializing the variables to make the while loop gathering tweets work without duplicating anything. 

In [2]:
min_tweet = 99999999999999999999999999999999999
firstpass = True

The search terms are the most basic NFL social media engagement ones. Each team has a Twitter handle and custom hash tag that displays an emoji relating to the team. Each game also has a 'neutral' hashtag for the matchup, just saying the two teams. That's a total of 5 terms in the full search bar, which I'll use to classify my tweets and retweets. 

In [3]:
while True:
    tweet_list = api.search(q='#%23GBvsNYG%20OR%20#%23GoPackGo%20OR%20#%23GiantsPrideOR%40packersOR%40giants', 
                            count = 100, tweet_mode = 'extended', max_id = min_tweet, result_type = 'recent')
    min_tweet = tweet_list[-1].id_str
    original_tweets = []
    retweets = []
    for tweet in tweet_list:
        try: #Using try/catch as an if/else statement. If it's a retweet, the try executes. Else, it's a tweet.
            retweet_dict = {'original_id': tweet.retweeted_status.id_str, 'original_tweeter': tweet.retweeted_status.user.id,
                            'original_name': tweet.retweeted_status.user.name, 'retweet_id': tweet.id_str, 
                            'retweeter': tweet.user.id, 'retweeter_name': tweet.user.name, 'retweet_time': tweet.created_at}
            retweets.append(retweet_dict)
            continue
        except:
            pass
        tweet_dict = {'tweet_id': tweet.id, 'text': tweet.full_text, 'tweeter': tweet.user.id, 
                      'tweeter_name': tweet.user.name, 'tweet_time': tweet.created_at}
        original_tweets.append(tweet_dict)

    if firstpass:
        dftweets = pd.DataFrame(original_tweets)
        dfretweets = pd.DataFrame(retweets)
        firstpass = False
    else:
        dftweets = dftweets.append(pd.DataFrame(original_tweets), ignore_index=True)
        dfretweets = dfretweets.append(pd.DataFrame(retweets), ignore_index=True)
    time.sleep(45) #make sure I can actually scrape tweets and avoid usage limits. I got time.
    if dftweets['tweet_time'].min() < datetime.strptime('2019-12-01T16::00', '%Y-%m-%dT%H::%M'):
        break

#Removing duplicates and exporting when all the tweets are gathered.
dftweets.drop_duplicates(inplace = True)
dfretweets.drop_duplicates(inplace = True)
dftweets.to_csv('packersgiantstweets.csv',index=False)
dfretweets.to_csv('packersgiantsretweets.csv',index=False)